In [ ]:
!pip install -q pyomo
!pip install -q folium
!apt update
!apt install -y -q coinor-cbc
!apt install -y -q glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Get:12 http:/

In [ ]:
import numpy as np
import pandas as pd
import folium
import math
import base64
from pyomo.environ import (ConcreteModel,
                           Var,
                           ConstraintList,
                           Objective,
                           Binary,
                           Reals,
                           PositiveIntegers,
                           SolverFactory)

# initialize solver
solver = SolverFactory("cbc", executable='/usr/bin/cbc')
solver_glpk = solver = SolverFactory('glpk')

In [ ]:
!pip install geopy
!pip install pandas


In [ ]:
import pandas as pd

# For tab-separated values
city_names = pd.read_csv('cities.txt', delimiter='\t', header=None, names=['City'])
# Get the first 5 cities
city_names = city_names.head(15)

In [ ]:
city_names

,City
0,IIT Bombay Powai Mumbai
1,Taj Mahal Agra Uttar Pradesh
2,Qutub Minar Delhi
3,Red Fort Complex Delhi
4,Humayun's Tomb Delhi
5,Jaipur City Rajasthan
6,Ranthambore Fort Rajasthan
7,Kumbhalgarh Fort Rajasthan
8,Kaziranga National Park Assam
9,Gateway of India Mumbai Maharashtra


In [ ]:
city_names.shape

(15, 1)

In [ ]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_locator")

def get_lat_lon(city_name):
    try:
        location = geolocator.geocode(city_name)
        return (location.latitude, location.longitude)
    except:
        return (None, None)

city_names['lat'], city_names['lon'] = zip(*city_names['City'].apply(get_lat_lon))

In [ ]:
city_names.to_csv('cities_data.csv', index=False)

In [ ]:
cities_data = pd.read_csv("cities_data.csv")
cities_data

,City,lat,lon
0,IIT Bombay Powai Mumbai,19.132619,72.914970
1,Taj Mahal Agra Uttar Pradesh,27.175007,78.042101
2,Qutub Minar Delhi,28.524413,77.185450
3,Red Fort Complex Delhi,28.657393,77.241274
4,Humayun's Tomb Delhi,28.593286,77.250647
5,Jaipur City Rajasthan,26.915458,75.818982
6,Ranthambore Fort Rajasthan,26.018569,76.455976
7,Kumbhalgarh Fort Rajasthan,25.152677,73.586285
8,Kaziranga National Park Assam,26.658105,93.392744
9,Gateway of India Mumbai Maharashtra,18.921966,72.834566


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Define the haversine function using math
def haversine(lon1, lat1, lon2, lat2):
    lon1 = math.radians(lon1)
    lon2 = math.radians(lon2)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    lon_diff = lon2 - lon1
    lat_diff = lat2 - lat1
    a = math.sin(lat_diff / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(lon_diff / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6357
    return c * r

In [ ]:
# Create a distance matrix
num_cities = len(cities_data)
distance_matrix = np.zeros((num_cities, num_cities))

for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            distance = haversine(cities_data.iloc[i]['lon'], cities_data.iloc[i]['lat'], cities_data.iloc[j]['lon'], cities_data.iloc[j]['lat'])
            distance_matrix[i, j] = round(distance, 1)
        else:
            distance_matrix[i, j] = 0  # Distance to itself is 0

# Convert distance matrix to DataFrame
cities_distances = pd.DataFrame(distance_matrix, index=cities_data['City'], columns=cities_data['City'])

# Save the DataFrame to a CSV file
cities_distances.to_csv('distance_matrix.csv')

In [ ]:
# Save the DataFrame to a CSV file
cities_distances.to_csv('cities_distances.csv')

In [ ]:
cities_distances

City,IIT Bombay Powai Mumbai,Taj Mahal Agra Uttar Pradesh,Qutub Minar Delhi,Red Fort Complex Delhi,Humayun's Tomb Delhi,Jaipur City Rajasthan,Ranthambore Fort Rajasthan,Kumbhalgarh Fort Rajasthan,Kaziranga National Park Assam,Gateway of India Mumbai Maharashtra,Konark Sun Temple Odisha,Hawa Mahal Jaipur Rajasthan,Golconda Fort Hyderabad Telangana,India Gate Delhi,Kashi Vishwanath Temple Varanasi Uttar Pradesh
City,,,,,,,,,,,,,,,
IIT Bombay Powai Mumbai,0.0,1034.0,1128.3,1144.0,1137.8,912.9,845.6,671.5,2249.7,24.8,1380.5,914.0,609.7,1139.0,1242.2
Taj Mahal Agra Uttar Pradesh,1034.0,0.0,171.7,182.2,175.5,221.6,203.0,497.2,1518.8,1058.5,1150.3,220.7,1087.0,178.3,535.9
Qutub Minar Delhi,1128.3,171.7,0.0,15.7,9.9,223.3,287.2,516.6,1605.8,1153.1,1314.6,222.1,1242.3,10.7,677.5
Red Fort Complex Delhi,1144.0,182.2,15.7,0.0,7.2,238.4,302.8,530.9,1601.4,1168.8,1321.3,237.2,1256.4,5.1,680.5
Humayun's Tomb Delhi,1137.8,175.5,9.9,7.2,0.0,233.3,296.2,526.5,1600.0,1162.6,1315.6,232.0,1249.3,3.0,676.0
Jaipur City Rajasthan,912.9,221.6,223.3,238.4,233.3,0.0,117.9,296.3,1739.4,937.7,1303.9,1.2,1090.3,233.8,738.1
Ranthambore Fort Rajasthan,845.6,203.0,287.2,302.8,296.2,117.9,0.0,302.8,1684.3,870.4,1196.1,118.3,978.8,297.8,660.1
Kumbhalgarh Fort Rajasthan,671.5,497.2,516.6,530.9,526.5,296.3,302.8,0.0,1981.8,695.6,1407.8,297.5,995.1,526.6,945.8
Kaziranga National Park Assam,2249.7,1518.8,1605.8,1601.4,1600.0,1739.4,1684.3,1981.8,0.0,2267.4,1056.6,1738.6,1851.2,1602.2,1045.9


In [ ]:
# cities_demands = pd.read_csv("cities_demands.csv", index_col=0)
# cities_demands.head()

In [ ]:
import folium
import math

def get_map(cities_data, cities_distances, polylines=None, distances=None):
    # Extract coordinates and names from cities_data
    cities_coords = cities_data.iloc[:, 1:3].to_numpy()
    cities_names = cities_data.iloc[:, 0].to_numpy()
    cities_dists = cities_distances.to_numpy()

    # Calculate map bounds
    min_pt = (cities_data[['lat', 'lon']].min().values).tolist()
    max_pt = (cities_data[['lat', 'lon']].max().values).tolist()

    # Create the map centered on a default location
    mymap = folium.Map(location=[18.9690, 72.8777])

    # Add markers for each city
    for name, pt in zip(cities_names, cities_coords):
        folium.Marker(list(pt), popup=name).add_to(mymap)

    # Fit map bounds
    mymap.fit_bounds([min_pt, max_pt])

    # Add polylines if provided
    if polylines is not None:
        for i in range(len(polylines)):
            line = polylines[i]
            # Calculate the bearing between the two points
            start_point = line[0]
            end_point = line[-1]
            lat1, lon1 = start_point
            lat2, lon2 = end_point

            # Convert latitude and longitude from degrees to radians
            lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
            lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)

            # Calculate bearing
            delta_lon = lon2_rad - lon1_rad
            x = math.sin(delta_lon) * math.cos(lat2_rad)
            y = math.cos(lat1_rad) * math.sin(lat2_rad) - (math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon))
            bearing = math.atan2(x, y)
            bearing = math.degrees(bearing)
            bearing = (bearing + 360) % 360  # Normalize bearing to 0-360 degrees

            # Calculate the midpoint of the polyline
            midpoint = [(line[0][0] + line[1][0]) / 2, (line[0][1] + line[1][1]) / 2]

            # Add polyline to the map
            if distances is not None:
                folium.PolyLine(line, tooltip=distances[i], popup=distances[i], weight=6, color="#111").add_to(mymap)
            else:
                folium.PolyLine(line, weight=1, color="#111").add_to(mymap)

            # Add an arrow marker to the end of the polyline with the correct rotation
            folium.RegularPolygonMarker(
                location=midpoint,
                color="#111",
                fill_color="#111",
                number_of_sides=3,
                opacity=1,
                radius=10,
                rotation=bearing - 90,  # Adjust for the polygon orientation
                tooltip=''
            ).add_to(mymap)

    return mymap


In [ ]:
mymap = get_map(cities_data, cities_distances)
mymap

In [ ]:
cities_coords = cities_data.iloc[:, 1:3].values
print(cities_coords)
cities_names = cities_data.iloc[:, 0].values
print(cities_names)
distance_matrix = cities_distances.to_numpy()
print("shape of distance matrix: ", distance_matrix.shape)

[[19.13261855 72.9149702 ]
 [27.1750075  78.04210126]
 [28.524413   77.18545014]
 [28.65739275 77.24127352]
 [28.5932856  77.2506468 ]
 [26.9154576  75.8189817 ]
 [26.01856915 76.45597609]
 [25.15267685 73.58628506]
 [26.65810465 93.39274406]
 [18.9219661  72.83456566]
 [19.8876735  86.0945357 ]
 [26.923932   75.8268652 ]
 [17.3833665  78.4010991 ]
 [28.6129332  77.22949282]
 [25.31082185 83.0106679 ]]
['IIT Bombay Powai Mumbai' 'Taj Mahal Agra Uttar Pradesh'
 'Qutub Minar Delhi' 'Red Fort Complex Delhi' "Humayun's Tomb Delhi"
 'Jaipur City Rajasthan' 'Ranthambore Fort Rajasthan'
 'Kumbhalgarh Fort Rajasthan' 'Kaziranga National Park Assam'
 'Gateway of India Mumbai Maharashtra' 'Konark Sun Temple Odisha'
 'Hawa Mahal Jaipur Rajasthan' 'Golconda Fort Hyderabad Telangana'
 'India Gate Delhi' 'Kashi Vishwanath Temple Varanasi Uttar Pradesh']
shape of distance matrix:  (15, 15)


In [ ]:
def PDTSP1Model(distance_matrix, depot_index):
    n = len(distance_matrix)


    model = ConcreteModel(name="1-PDTSP")

    model.x = Var(range(n), range(n), initialize=0.0, domain=Binary)
    model.u = Var(range(n), domain=PositiveIntegers)

    model.objective = Objective(expr=sum(distance_matrix[i, j]*model.x[i, j] for i in range(n) for j in range(n)))

    model.constraints = ConstraintList()
    for i in range(n):
        model.constraints.add(sum(model.x[i, k] for k in range(n) if i != k) == 1)

    for j in range(n):
        model.constraints.add(sum(model.x[k, j] for k in range(n) if j != k) == 1)

    model.constraints.add(model.u[0] == 1)
    for i in range(1, n):
        model.u[i].setlb(2)
        model.u[i].setub(n)

    for i in range(n):
        for j in range(n):
            if i != 0 and j != 0:
                model.constraints.add(model.u[i] - model.u[j] + 1 <= (n-1)*(1-model.x[i, j]))

    return model

In [19]:
#model = PDTSP1Model(distance_matrix, demand_list, truck_capacity, depot_index=0)
model = PDTSP1Model(distance_matrix, depot_index=0)
model.pprint()

2 Var Declarations
    u : Size=15, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :  None : False :  True : PositiveIntegers
          1 :     2 :  None :    15 : False :  True : PositiveIntegers
          2 :     2 :  None :    15 : False :  True : PositiveIntegers
          3 :     2 :  None :    15 : False :  True : PositiveIntegers
          4 :     2 :  None :    15 : False :  True : PositiveIntegers
          5 :     2 :  None :    15 : False :  True : PositiveIntegers
          6 :     2 :  None :    15 : False :  True : PositiveIntegers
          7 :     2 :  None :    15 : False :  True : PositiveIntegers
          8 :     2 :  None :    15 : False :  True : PositiveIntegers
          9 :     2 :  None :    15 : False :  True : PositiveIntegers
         10 :     2 :  None :    15 : False :  True : PositiveIntegers
         11 :     2 :  None :    15 : False :  True : PositiveInt

In [20]:
print("Problem Details")
print("."*100)
print("Number of Cities: ", len(distance_matrix))
print("="*100)
print("Model Details: ")
print("."*100)
print("Total Number of variables: ", len(model.x) + len(model.u))
print("Total Number of constraints: ", len(model.constraints) + 2*len(model.u) + len(model.x))

Problem Details
....................................................................................................
Number of Cities:  15
Model Details: 
....................................................................................................
Total Number of variables:  240
Total Number of constraints:  482


In [21]:
%%time
## CBC Solver
result = solver.solve(model)
print("Solver Status: ", result.solver.status)
print("Solver Termination Condition: ", result.solver.termination_condition)

Solver Status:  ok
Solver Termination Condition:  optimal
CPU times: user 38 ms, sys: 3.9 ms, total: 41.9 ms
Wall time: 1.08 s


In [22]:
print("optimal objective value: ", model.objective())

optimal objective value:  5643.5


In [23]:
sol_edges = []
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        if model.x[i, j].value:
            sol_edges.append((j, i))

In [24]:
polylines = []
for edge in sol_edges:
    inp, out = edge
    inp_coordinate = cities_coords[inp]
    out_coordinate = cities_coords[out]
    polylines.append((inp_coordinate, out_coordinate))

In [25]:
sol_dict = {key:val for key, val in sol_edges}
optimal_path_idx = []
c = 0
idx = 0
while c < len(distance_matrix):
    optimal_path_idx.append(idx)
    idx = sol_dict[idx]
    c += 1
optimal_path_idx.append(0)

In [26]:
optimal_path = [cities_names[i] for i in optimal_path_idx]
for city in optimal_path:
    print(city, " -> ", end="")

IIT Bombay Powai Mumbai  -> Gateway of India Mumbai Maharashtra  -> Golconda Fort Hyderabad Telangana  -> Konark Sun Temple Odisha  -> Kaziranga National Park Assam  -> Kashi Vishwanath Temple Varanasi Uttar Pradesh  -> Taj Mahal Agra Uttar Pradesh  -> Humayun's Tomb Delhi  -> Red Fort Complex Delhi  -> India Gate Delhi  -> Qutub Minar Delhi  -> Hawa Mahal Jaipur Rajasthan  -> Jaipur City Rajasthan  -> Ranthambore Fort Rajasthan  -> Kumbhalgarh Fort Rajasthan  -> IIT Bombay Powai Mumbai  -> 

In [27]:
distances = []
for i in range(len(optimal_path_idx)-1):
    distances.append(f"{distance_matrix[optimal_path_idx[i], optimal_path_idx[i+1]] :.2f} K.M.")

In [28]:
mymap = get_map(cities_data, cities_distances, polylines, distances)
mymap.save("map2.html")
mymap